In [2]:
import requests
import pandas as pd
import numpy as np

url = "https://fantasy.premierleague.com/api/bootstrap-static/"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print("Nombre de joueurs récupérés :", len(data["elements"]))
    # Exemple : afficher le nom du premier joueur
    print("Premier joueur :", data["elements"][0]["web_name"])
else:
    print("Erreur lors de la requête :", response.status_code)

Nombre de joueurs récupérés : 740
Premier joueur : Raya


In [3]:
fixtures_url = "https://fantasy.premierleague.com/api/fixtures/"
fixtures_response = requests.get(fixtures_url)
fixtures = fixtures_response.json()
print("Nombre de matchs à venir :", len(fixtures))

Nombre de matchs à venir : 380


In [4]:
# Extraire la liste des joueurs (éléments)
players = data['elements']
# Convertir en DataFrame pandas
df_players = pd.DataFrame(players)
# Convertir listes d'équipes et de positions en DataFrames
df_teams = pd.DataFrame(data['teams'])[["id", "name", "short_name"]]
df_positions = pd.DataFrame(data['element_types'])[["id", "singular_name_short"]]

In [5]:
# Joindre le nom de l'équipe sur la colonne 'team' (qui contient l'ID)
df_players = df_players.merge(df_teams, left_on='team', right_on='id', suffixes=('', '_team'))

# Joindre la position du joueur sur la colonne 'element_type' (ID)
df_players = df_players.merge(df_positions, left_on='element_type', right_on='id', suffixes=('', '_pos'))

# Sélectionner les colonnes intéressantes pour plus de lisibilité
df_final = df_players[[
    'first_name', 'second_name', 'total_points', 'now_cost', 'minutes', 
    'name', 'short_name',  # équipe
    'singular_name_short'  # position
]]

# Renommer pour clarté
df_final = df_final.rename(columns={
    'name': 'team_name',
    'short_name': 'team_short',
    'singular_name_short': 'position',
    'now_cost': 'price'
})

In [6]:
# Afficher un aperçu trié par points totaux décroissants
df_final_sorted = df_final.sort_values(by='total_points', ascending=False)


In [7]:
df_final_sorted['points_per_price'] = df_final_sorted['total_points']/df_final_sorted['price']
df_final_sorted = df_final_sorted.sort_values(by='points_per_price', ascending=False)

In [8]:
df_final_sorted

,first_name,second_name,total_points,price,minutes,team_name,team_short,position,points_per_price
279,Marc,Guéhi,29,46,360,Crystal Palace,CRY,DEF,0.630435
124,Marcos,Senesi Barón,28,46,360,Bournemouth,BOU,DEF,0.608696
625,Robin,Roefs,27,45,360,Sunderland,SUN,GKP,0.600000
7,Jurriën,Timber,34,57,262,Arsenal,ARS,DEF,0.596491
244,Trevoh,Chalobah,30,51,338,Chelsea,CHE,DEF,0.588235
...,...,...,...,...,...,...,...,...,...
731,Fábio,Soares Silva,0,49,0,Wolves,WOL,FWD,0.000000
733,Leon,Chiwome,0,45,0,Wolves,WOL,FWD,0.000000
734,Nathan,Fraser,0,44,0,Wolves,WOL,FWD,0.000000
725,Boubacar,Traoré,0,50,0,Wolves,WOL,MID,0.000000


In [9]:
df_teams = pd.DataFrame(data['teams'])[["id", "name", "short_name"]]

# Récupérer les fixtures
fixtures_url = "https://fantasy.premierleague.com/api/fixtures/"
response_fixtures = requests.get(fixtures_url)
fixtures = response_fixtures.json()

df_fixtures = pd.DataFrame(fixtures)

# Pour chaque match, récupérer les noms d'équipes à domicile et à l'extérieur
df_fixtures = df_fixtures.merge(df_teams, left_on='team_h', right_on='id', suffixes=('', '_home'))
df_fixtures = df_fixtures.merge(df_teams, left_on='team_a', right_on='id', suffixes=('', '_away'))

# Construire un tableau avec équipes et degré de difficulté pour domicile et extérieur
df_fixtures_readable = df_fixtures[[
    'event',  # numéro de la journée/matchweek
    'kickoff_time',
    'name',        # équipe à domicile
    'team_h_difficulty',
    'name_away',   # équipe à l'extérieur
    'team_a_difficulty',
    'finished',
    'team_h_score',
    'team_a_score'
]]

df_fixtures_readable = df_fixtures_readable.rename(columns={
    'name': 'home_team',
    'name_away': 'away_team',
    'team_h_difficulty': 'home_difficulty',
    'team_a_difficulty': 'away_difficulty',
    'event': 'gameweek'
})

# Affichage des premières lignes
df_fixtures_readable


,gameweek,kickoff_time,home_team,home_difficulty,away_team,away_difficulty,finished,team_h_score,team_a_score
0,1,2025-08-15T19:00:00Z,Liverpool,3,Bournemouth,5,True,4.0,2.0
1,1,2025-08-16T11:30:00Z,Aston Villa,3,Newcastle,4,True,0.0,0.0
2,1,2025-08-16T14:00:00Z,Brighton,3,Fulham,3,True,1.0,1.0
3,1,2025-08-16T14:00:00Z,Spurs,2,Burnley,3,True,3.0,0.0
4,1,2025-08-16T14:00:00Z,Sunderland,2,West Ham,2,True,3.0,0.0
...,...,...,...,...,...,...,...,...,...
375,38,2026-05-24T15:00:00Z,Man City,3,Aston Villa,4,False,NaN,NaN
376,38,2026-05-24T15:00:00Z,Nott'm Forest,3,Bournemouth,3,False,NaN,NaN
377,38,2026-05-24T15:00:00Z,Spurs,2,Everton,3,False,NaN,NaN
378,38,2026-05-24T15:00:00Z,Sunderland,3,Chelsea,2,False,NaN,NaN


In [10]:
# Filtrer les fixtures pour la Gameweek 4
gw4_fixtures = df_fixtures_readable[df_fixtures_readable['gameweek'] == 4]
gw4_fixtures





,gameweek,kickoff_time,home_team,home_difficulty,away_team,away_difficulty,finished,team_h_score,team_a_score
30,4,2025-09-13T11:30:00Z,Arsenal,3,Nott'm Forest,4,True,3.0,0.0
31,4,2025-09-13T14:00:00Z,Bournemouth,3,Brighton,3,True,2.0,1.0
32,4,2025-09-13T14:00:00Z,Crystal Palace,2,Sunderland,3,True,0.0,0.0
33,4,2025-09-13T14:00:00Z,Everton,3,Aston Villa,3,True,0.0,0.0
34,4,2025-09-13T14:00:00Z,Fulham,2,Leeds,3,True,1.0,0.0
35,4,2025-09-13T14:00:00Z,Newcastle,2,Wolves,4,True,1.0,0.0
36,4,2025-09-13T16:30:00Z,West Ham,3,Spurs,2,True,0.0,3.0
37,4,2025-09-13T19:00:00Z,Brentford,3,Chelsea,3,True,2.0,2.0
38,4,2025-09-14T13:00:00Z,Burnley,4,Liverpool,2,True,0.0,1.0
39,4,2025-09-14T15:30:00Z,Man City,3,Man Utd,4,True,3.0,0.0


In [11]:
df_fixtures_readable[(df_fixtures_readable['home_team'] == 'Arsenal') | (df_fixtures_readable['away_team'] == 'Arsenal')]


,gameweek,kickoff_time,home_team,home_difficulty,away_team,away_difficulty,finished,team_h_score,team_a_score
8,1,2025-08-17T15:30:00Z,Man Utd,4,Arsenal,3,True,0.0,1.0
15,2,2025-08-23T16:30:00Z,Arsenal,2,Leeds,4,True,5.0,0.0
28,3,2025-08-31T15:30:00Z,Liverpool,4,Arsenal,5,True,1.0,0.0
30,4,2025-09-13T11:30:00Z,Arsenal,3,Nott'm Forest,4,True,3.0,0.0
49,5,2025-09-21T15:30:00Z,Arsenal,4,Man City,4,False,NaN,NaN
58,6,2025-09-28T15:30:00Z,Newcastle,4,Arsenal,4,False,NaN,NaN
62,7,2025-10-04T14:00:00Z,Arsenal,2,West Ham,4,False,NaN,NaN
76,8,2025-10-18T16:30:00Z,Fulham,4,Arsenal,3,False,NaN,NaN
85,9,2025-10-26T14:00:00Z,Arsenal,3,Crystal Palace,4,False,NaN,NaN
91,10,2025-11-01T15:00:00Z,Burnley,4,Arsenal,2,False,NaN,NaN


In [12]:
# Préparer les DataFrames domicile et extérieur pour calculer la moyenne difficulté
home_df = df_fixtures[['event', 'team_h', 'team_h_difficulty']].copy()
home_df = home_df.rename(columns={'team_h': 'team_id', 'team_h_difficulty': 'difficulty'})

away_df = df_fixtures[['event', 'team_a', 'team_a_difficulty']].copy()
away_df = away_df.rename(columns={'team_a': 'team_id', 'team_a_difficulty': 'difficulty'})

all_fixtures = pd.concat([home_df, away_df], ignore_index=True)

# Définir la gameweek courante et fenêtre de 5 GW
current_gw = 4
window = 5

# Filtrer les prochaines 5 GW
next_fixtures = all_fixtures[
    (all_fixtures['event'] > current_gw) & (all_fixtures['event'] <= current_gw + window)
]

# Calculer la moyenne difficulté par équipe
mean_difficulty = next_fixtures.groupby('team_id')['difficulty'].mean().reset_index()
mean_difficulty = mean_difficulty.rename(columns={'difficulty': 'avg_next_5gw_difficulty'})

# Préparer un DataFrame lisible des fixtures avec noms équipes
df_fixtures_readable = df_fixtures.merge(df_teams, left_on='team_h', right_on='id', suffixes=('', '_home'))
df_fixtures_readable = df_fixtures_readable.merge(df_teams, left_on='team_a', right_on='id', suffixes=('', '_away'))

# Sélectionner et renommer
df_fixtures_readable = df_fixtures_readable[[
    'event', 'kickoff_time',
    'team_h', 'name', 'team_h_difficulty',
    'team_a', 'name_away', 'team_a_difficulty',
    'finished', 'team_h_score', 'team_a_score'
]]

df_fixtures_readable = df_fixtures_readable.rename(columns={
    'event': 'gameweek',
    'name': 'home_team',
    'team_h_difficulty': 'home_difficulty',
    'name_away': 'away_team',
    'team_a_difficulty': 'away_difficulty'
})

# Fusionner la moyenne de difficulté sur IDs d'équipe domicile et extérieur
df_fixtures_readable = df_fixtures_readable.merge(
    mean_difficulty, left_on='team_h', right_on='team_id', how='left'
).rename(columns={'avg_next_5gw_difficulty': 'home_avg_difficulty'}).drop(columns=['team_id'])

df_fixtures_readable = df_fixtures_readable.merge(
    mean_difficulty, left_on='team_a', right_on='team_id', how='left'
).rename(columns={'avg_next_5gw_difficulty': 'away_avg_difficulty'}).drop(columns=['team_id'])

# Afficher un aperçu
print(df_fixtures_readable.head(10))




   gameweek          kickoff_time  team_h      home_team  home_difficulty  \
0         1  2025-08-15T19:00:00Z      12      Liverpool                3   
1         1  2025-08-16T11:30:00Z       2    Aston Villa                3   
2         1  2025-08-16T14:00:00Z       6       Brighton                3   
3         1  2025-08-16T14:00:00Z      18          Spurs                2   
4         1  2025-08-16T14:00:00Z      17     Sunderland                2   
5         1  2025-08-16T16:30:00Z      20         Wolves                4   
6         1  2025-08-17T13:00:00Z       7        Chelsea                3   
7         1  2025-08-17T13:00:00Z      16  Nott'm Forest                3   
8         1  2025-08-17T15:30:00Z      14        Man Utd                4   
9         1  2025-08-18T19:00:00Z      11          Leeds                2   

   team_a       away_team       away_team  away_difficulty  finished  \
0       4     Bournemouth     Bournemouth                5      True   
1      1

In [13]:
df_fixtures_readable


,gameweek,kickoff_time,team_h,home_team,home_difficulty,team_a,away_team,away_team,away_difficulty,finished,team_h_score,team_a_score,home_avg_difficulty,away_avg_difficulty
0,1,2025-08-15T19:00:00Z,12,Liverpool,3,4,Bournemouth,Bournemouth,5,True,4.0,2.0,3.0,2.8
1,1,2025-08-16T11:30:00Z,2,Aston Villa,3,15,Newcastle,Newcastle,4,True,0.0,0.0,2.8,3.2
2,1,2025-08-16T14:00:00Z,6,Brighton,3,10,Fulham,Fulham,3,True,1.0,1.0,3.2,3.6
3,1,2025-08-16T14:00:00Z,18,Spurs,2,3,Burnley,Burnley,3,True,3.0,0.0,2.6,3.2
4,1,2025-08-16T14:00:00Z,17,Sunderland,2,19,West Ham,West Ham,2,True,3.0,0.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,38,2026-05-24T15:00:00Z,13,Man City,3,2,Aston Villa,Aston Villa,4,False,NaN,NaN,3.0,2.8
376,38,2026-05-24T15:00:00Z,16,Nott'm Forest,3,4,Bournemouth,Bournemouth,3,False,NaN,NaN,2.8,2.8
377,38,2026-05-24T15:00:00Z,18,Spurs,2,9,Everton,Everton,3,False,NaN,NaN,2.6,3.4
378,38,2026-05-24T15:00:00Z,17,Sunderland,3,7,Chelsea,Chelsea,2,False,NaN,NaN,3.0,3.0


In [14]:
df_fixtures.head(50)

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,...,stats,team_h_difficulty,team_a_difficulty,pulse_id,id_home,name,short_name,id_away,name_away,short_name_away
0,2561895,1,True,True,1,2025-08-15T19:00:00Z,90,False,True,4,...,"[{'identifier': 'goals_scored', 'a': [{'value'...",3,5,124791,12,Liverpool,LIV,4,Bournemouth,BOU
1,2561896,1,True,True,2,2025-08-16T11:30:00Z,90,False,True,15,...,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",3,4,124792,2,Aston Villa,AVL,15,Newcastle,NEW
2,2561897,1,True,True,3,2025-08-16T14:00:00Z,90,False,True,10,...,"[{'identifier': 'goals_scored', 'a': [{'value'...",3,3,124793,6,Brighton,BHA,10,Fulham,FUL
3,2561900,1,True,True,6,2025-08-16T14:00:00Z,90,False,True,3,...,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2,3,124796,18,Spurs,TOT,3,Burnley,BUR
4,2561899,1,True,True,5,2025-08-16T14:00:00Z,90,False,True,19,...,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2,2,124795,17,Sunderland,SUN,19,West Ham,WHU
5,2561901,1,True,True,7,2025-08-16T16:30:00Z,90,False,True,13,...,"[{'identifier': 'goals_scored', 'a': [{'value'...",4,3,124797,20,Wolves,WOL,13,Man City,MCI
6,2561902,1,True,True,8,2025-08-17T13:00:00Z,90,False,True,8,...,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",3,4,124798,7,Chelsea,CHE,8,Crystal Palace,CRY
7,2561898,1,True,True,4,2025-08-17T13:00:00Z,90,False,True,5,...,"[{'identifier': 'goals_scored', 'a': [{'value'...",3,3,124794,16,Nott'm Forest,NFO,5,Brentford,BRE
8,2561903,1,True,True,9,2025-08-17T15:30:00Z,90,False,True,1,...,"[{'identifier': 'goals_scored', 'a': [{'value'...",4,3,124799,14,Man Utd,MUN,1,Arsenal,ARS
9,2561904,1,True,True,10,2025-08-18T19:00:00Z,90,False,True,9,...,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2,2,124800,11,Leeds,LEE,9,Everton,EVE
